## Deliverable 2. Create a Customer Travel Destinations Map.

In [46]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [47]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Alta,AR,-38.8759,-62.0736,64.20,34,1,16.87,clear sky
1,1,Hermanus,ZA,-34.4187,19.2345,53.80,61,0,5.82,clear sky
2,2,Cidreira,BR,-30.1811,-50.2056,79.61,73,10,8.84,clear sky
3,3,Lebu,CL,-37.6167,-73.6500,54.72,61,39,10.22,scattered clouds
4,4,Fairbanks,US,64.8378,-147.7164,48.20,43,0,4.61,clear sky


In [48]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [49]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Teahupoo,PF,-17.8333,-149.2667,76.96,77,100,25.50,light rain
12,12,Rikitea,PF,-23.1203,-134.9692,77.16,72,7,24.67,clear sky
16,16,Bolobo,CD,-2.1667,16.2333,75.56,80,94,1.50,overcast clouds
26,26,Isangel,VU,-19.5500,169.2667,76.57,78,100,13.35,overcast clouds
56,56,Thanh Hoa,VN,19.8000,105.7667,76.75,98,66,4.54,broken clouds
73,73,Nabire,ID,-3.3667,135.4833,75.22,92,94,3.58,overcast clouds
116,116,Manado,ID,1.4870,124.8455,77.77,94,40,1.14,scattered clouds
131,131,Marihatag,PH,8.7936,126.2969,77.29,90,99,1.63,light rain
137,137,Langsa,ID,4.4683,97.9683,76.28,97,98,3.00,overcast clouds
142,142,Kahului,US,20.8947,-156.4700,77.02,78,40,23.02,light rain


In [50]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                54
City                   54
Country                54
Lat                    54
Lng                    54
Max Temp               54
Humidity               54
Cloudiness             54
Wind Speed             54
Current Description    54
dtype: int64

In [51]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_nonull_df = preferred_cities_df.dropna()
preferred_cities_nonull_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [52]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_nonull_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Teahupoo,PF,76.96,light rain,-17.8333,-149.2667,
12,Rikitea,PF,77.16,clear sky,-23.1203,-134.9692,
16,Bolobo,CD,75.56,overcast clouds,-2.1667,16.2333,
26,Isangel,VU,76.57,overcast clouds,-19.5500,169.2667,
56,Thanh Hoa,VN,76.75,broken clouds,19.8000,105.7667,
73,Nabire,ID,75.22,overcast clouds,-3.3667,135.4833,
116,Manado,ID,77.77,scattered clouds,1.4870,124.8455,
131,Marihatag,PH,77.29,light rain,8.7936,126.2969,
137,Langsa,ID,76.28,overcast clouds,4.4683,97.9683,
142,Kahului,US,77.02,light rain,20.8947,-156.4700,


In [53]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
         print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [54]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [55]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Database.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [56]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [57]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))